In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
import polars as pl
import ast
import time
import glob
import numpy as np
from scipy.special import erf
from rtseg.utils.param_io import load_params
from pathlib import Path
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread
from rtseg.forkplot import compute_forkplot_stats
from rtseg.cells.plotting import generate_abins_lbins, generate_fork_plot, get_bulk_init_area, slice_fork_plot_around_init
from rtseg.cells.plotting import fast_generate_fork_plot
import matplotlib.patches as patches
from numba import jit, vectorize
import glob
import os
%matplotlib qt5

In [3]:
from scipy.optimize import curve_fit

In [4]:
params_path = Path("/home/pk/Documents//rtseg/rtseg/resources/reference_params/reference_microscope.yaml")
param = load_params(params_path, ref_type='expt')

In [5]:
save_dir = Path("/mnt/sda1/REALTIME/analysis/EXP-25-CD0117/")

In [8]:

if os.name == 'nt':
    use_pyarrow = True
else:
    use_pyarrow = False

fork_filenames = glob.glob(str(save_dir / Path('Pos[0-9]*/forks.parquet')))
columns_to_extract = ['area', 'length', 'normalized_internal_x', 'normalization_counts']
data = pl.read_parquet(fork_filenames, use_pyarrow=use_pyarrow, columns=columns_to_extract)

areas = data['area'].to_numpy()
lengths = data['length'].to_numpy()
longs = data['normalized_internal_x'].to_numpy()
counts = data['normalization_counts'].to_numpy()


bin_scale = param.Forkplots.bin_scale
heatmap_threshold = param.Forkplots.heatmap_threshold
pixel_size = param.Forkplots.pixel_size
min_length = param.Forkplots.min_length
arb_div_area = param.Forkplots.arb_div_area
init_area_cv = param.Forkplots.init_area_cv 

heatmap, mean_cell_lengths, abins, lbins, extent = generate_fork_plot(areas, lengths, longs, counts,
            bin_scale=bin_scale,
            pixel_size=pixel_size,
            heatmap_threshold=heatmap_threshold)

init_area = get_bulk_init_area(areas, counts, longs, lengths, min_length=min_length, 
                           pixel_size=pixel_size, arb_div_area=arb_div_area)

#area_bins_around_init, lbins_around_init, heatmap_around_init, mean_cell_lengths_around_init, abins_inds_around_init, lbins_inds_around_init = slice_fork_plot_around_init(abins, lbins, heatmap, mean_cell_lengths, init_area, init_area_cv)


In [9]:
init_area

1.4930643001248802

### Random tests

In [10]:
save_dir = Path("/mnt/sda1/REALTIME/analysis/EXP-25-CD0117/")

In [13]:
a = sorted(list(save_dir.glob('Pos*')))

In [15]:
a[0].name

'Pos1'